In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import svd, inv, eig

def wczytaj_i_wyczysc(nazwa_pliku):
    print(f"Wczytywanie {nazwa_pliku}...")
    df = pd.read_csv(nazwa_pliku, sep=';', decimal=',', header=None)
    df = df.dropna(axis=1, how='all')

    if np.allclose(df.iloc[:, 0], np.arange(1, len(df) + 1)):
        df = df.iloc[:, 1:]

    return df.values.astype(float).T

#1. WCZYTYWANIE I DOPASOWANIE 
try:
    X_raw = wczytaj_i_wyczysc('War5_X.csv')
    X_prime_raw = wczytaj_i_wyczysc('War5_Xprime.csv')
    
    # Sprawdzenie i wyrównanie wymiarów
    min_rows = min(X_raw.shape[0], X_prime_raw.shape[0])
    min_cols = min(X_raw.shape[1], X_prime_raw.shape[1])
    
    X = X_raw[:min_rows, :min_cols]
    X_prime = X_prime_raw[:min_rows, :min_cols]
    
    print(f"Ostateczny wymiar macierzy: {X.shape}")
    if X_raw.shape != X_prime_raw.shape:
        print(f"Uwaga: Skorygowano wymiary. Oryginalnie: X={X_raw.shape}, X'={X_prime_raw.shape}")

except Exception as e:
    print(f"Błąd krytyczny: {e}")
    exit()

# 2. ALGORYTM DMD 

# Krok 1: SVD
U, s, Vh = svd(X, full_matrices=False)
V = Vh.conj().T
Sigma = np.diag(s)

# Wybór rangi
r = len(s)
Ur = U[:, :r]
Sigmar = Sigma[:r, :r]
Vr = V[:, :r]

#Macierz A (Przybliżenie liniowe)
try:
    # Obliczamy macierz przejścia A
    # To jest rozwiązanie głównego zadania (X' ≈ AX)
    A = X_prime @ Vr @ inv(Sigmar) @ Ur.conj().T
    
    # Macierz zredukowana A_tilde
    A_tilde = Ur.conj().T @ X_prime @ Vr @ inv(Sigmar)
    
    print("Obliczenia zakończone sukcesem.")
except ValueError as e:
    print(f"Błąd mnożenia macierzy: {e}")
    print(f"Kształty: X_prime={X_prime.shape}, Vr={Vr.shape}, invSigmar={Sigmar.shape}, Ur.T={Ur.T.shape}")
    exit()

#Spektra
wartosci, W = eig(A_tilde)
Phi = X_prime @ Vr @ inv(Sigmar) @ W

#3. WYNIKI 
np.set_printoptions(precision=2, suppress=False)

print("\n" + "="*40)
print("WYNIKI DLA WARIANTRU 5")
print("="*40)
print(f"Macierz A (fragment 3x3):\n{A[:3, :3]}")
print(f"\nWartości własne (pierwsze 3):\n{wartosci[:3]}")

# Zapis do pliku
pd.DataFrame(A).to_csv('Wynik_A_Wariant5.csv', sep=';', decimal=',')
print("\nPełna macierz A została zapisana do: Wynik_A_Wariant5.csv")

Wczytywanie War5_X.csv...
Wczytywanie War5_Xprime.csv...
Ostateczny wymiar macierzy: (35, 23)
Uwaga: Skorygowano wymiary. Oryginalnie: X=(35, 23), X'=(36, 23)
Obliczenia zakończone sukcesem.

WYNIKI DLA WARIANTRU 5
Macierz A (fragment 3x3):
[[ 2.77e-22 -6.06e-15 -4.04e-14]
 [ 3.67e-22 -1.01e-14  2.14e-14]
 [-1.62e-22 -6.31e-15  8.39e-14]]

Wartości własne (pierwsze 3):
[ 2.23e-03+0.j  1.04e-07+0.j -4.32e-11+0.j]

Pełna macierz A została zapisana do: Wynik_A_Wariant5.csv
